# APT model: Famma-Macbeth Regression

In [1]:
from rqdata_utils import *
import pandas
import numpy as np
import scipy as sp
import alphalens as al
%matplotlib inline

## Loading Data

In [2]:
price_df,instrument_df,equity_df = get_price_instrument_equity("cn_stock_price_2012_2018.csv","cn_instrument_info_2012_2018.csv","cn_equity_daily_2012_2018.csv","sectorCode")

In [3]:
equity_df.head()

return   close  total_turnover      volume  \
date       order_book_id                                                 
2012-01-04 000001.XSHE   -0.027582  4.9886    2.275637e+08  40894428.0   
           000002.XSHE   -0.018742  5.6004    3.559891e+08  47432958.0   
           000004.XSHE   -0.022250  7.9100    3.763833e+06    465469.0   
           000005.XSHE    0.000000  3.8600    0.000000e+00         0.0   
           000006.XSHE   -0.009756  2.5072    7.619286e+06   2513811.0   

                            week   month report_quarter  pb_ratio  \
date       order_book_id                                            
2012-01-04 000001.XSHE    0.5242  0.4331            NaN       NaN   
           000002.XSHE    0.3887  0.4030         2011q3    1.5216   
           000004.XSHE    0.5797  0.7506         2011q3    8.8175   
           000005.XSHE       NaN     NaN         2011q3    5.3480   
           000006.XSHE    0.1601  0.1667         2011q3    1.4348   

                            net_profit  a_share_market_val_2  ps_ratio  \
date       order_book_id                                                 
2012-01-04 000001.XSHE             NaN                   NaN       NaN   
           000002.XSHE    4.106349e+09          7.082120e+10    1.1227   
           000004.XSHE    4.500363e+06          6.634549e+08    8.9157   
           000005.XSHE    1.365665e+07          3.527048e+09   55.5494   
           000006.XSHE    2.763917e+08          3.929464e+09    1.5507   

                          cash_received_from_sales_of_goods    market_cap  \
date       order_book_id                                                    
2012-01-04 000001.XSHE                                  NaN           NaN   
           000002.XSHE                         7.516785e+10  8.059489e+10   
           000004.XSHE                         5.949968e+07  6.642556e+08   
           000005.XSHE                         2.565851e+07  3.529328e+09   
           000006.XSHE                         2.531436e+09  4.015370e+09   

                           sectorCode  
date       order_book_id               
2012-01-04 000001.XSHE     Financials  
           000002.XSHE     RealEstate  
           000004.XSHE     HealthCare  
           000005.XSHE    Industrials  
           000006.XSHE     RealEstate

In [4]:
healthcareUniverse = instrument_df.index[instrument_df.sectorCode=='HealthCare'].values
len(healthcareUniverse)

172

In [5]:
def equity_universe_filtering(equity_df, universe):
    universeFilter = [book_id in set(universe) for book_id in equity_df.index.get_level_values(level=1).values]
    return equity_df[universeFilter]

In [6]:
healthcare_equity_df = equity_universe_filtering(equity_df, healthcareUniverse)
healthcare_equity_df.head()

return    close  total_turnover     volume  \
date       order_book_id                                                 
2012-01-04 000004.XSHE   -0.022250   7.9100      3763832.88   465469.0   
           000028.XSHE   -0.045433  19.5320      9326924.28   450553.0   
           000150.XSHE   -0.030295   1.6133      3109304.50  1867096.0   
           000153.XSHE   -0.028053   5.6757      9673054.49  1596020.0   
           000403.XSHE    0.000000   3.1625            0.00        0.0   

                            week   month report_quarter  pb_ratio  \
date       order_book_id                                            
2012-01-04 000004.XSHE    0.5797  0.7506         2011q3    8.8175   
           000028.XSHE    0.4291  0.2722         2011q3    4.3493   
           000150.XSHE    0.3647  0.3610         2011q3    1.4763   
           000153.XSHE    0.8331  2.4594         2011q3    2.1169   
           000403.XSHE       NaN     NaN         2011q3    5.4237   

                            net_profit  a_share_market_val_2  ps_ratio  \
date       order_book_id                                                 
2012-01-04 000004.XSHE    4.500363e+06          6.634549e+08    8.9157   
           000028.XSHE    2.481834e+08          4.753820e+09    0.3876   
           000150.XSHE    3.657858e+06          1.036800e+09    9.8634   
           000153.XSHE    1.560397e+07          1.360856e+09    1.0068   
           000403.XSHE    1.043543e+07          5.135403e+08    2.3231   

                          cash_received_from_sales_of_goods    market_cap  \
date       order_book_id                                                    
2012-01-04 000004.XSHE                         5.949968e+07  6.642556e+08   
           000028.XSHE                         1.053298e+10  5.872485e+09   
           000150.XSHE                         4.913279e+07  1.036800e+09   
           000153.XSHE                         1.329425e+09  1.531454e+09   
           000403.XSHE                         3.813537e+08  1.071118e+09   

                          sectorCode  
date       order_book_id              
2012-01-04 000004.XSHE    HealthCare  
           000028.XSHE    HealthCare  
           000150.XSHE    HealthCare  
           000153.XSHE    HealthCare  
           000403.XSHE    HealthCare

In [7]:
print("universe ratio: {}%".format(len(healthcare_equity_df)/len(equity_df)*100))

universe ratio: 7.367776097902626%


### benchmark

In [8]:
benchmark_df = pd.read_csv("cn_SH_healthcare_index_2012_2018.csv",names=['date','value'])
benchmark_df = benchmark_df.set_index('date',drop=True)

FileNotFoundError: [Errno 2] File b'cn_SH_healthcare_index_2012_2018.csv' does not exist: b'cn_SH_healthcare_index_2012_2018.csv'

In [ ]:
benchmark_df['return'] = np.log(benchmark_df.shift(1)/benchmark_df).fillna(0)
benchmark_df.head()

## Factor Returns

In [ ]:
def equity_factor_return(equity_df, factorColumn, nAllocations, longTop=True):
    equity_copy = equity_df.copy()
#     equity_copy["{}_rank".format(factorColumn)] = equity_copy.groupby(level='date')[factorColumn].rank()
#     equity_copy[equity_copy.groupby(level='date')[factorColumn].nlargest(nAllocations).index]["biggest_{}_{}".format(nAllocations,factorColumn)]=True
    largest = equity_copy[factorColumn].groupby(level='date').nlargest(nAllocations).reset_index(level=0,drop=True)
    smallest = equity_copy[factorColumn].groupby(level='date').nsmallest(nAllocations).reset_index(level=0,drop=True)
    r_largest = equity_copy.loc[largest.index,'return'].groupby(level='date').mean()
    r_smallest = equity_copy.loc[smallest.index,'return'].groupby(level='date').mean()
    LMS = r_largest - r_smallest
    if(longTop):
        return LMS
    else:
        return -LMS

In [ ]:
SMB = equity_factor_return(healthcare_equity_df, 'market_cap', 20,longTop=False)
SMB.head()

In [ ]:
HML = equity_factor_return(healthcare_equity_df, 'pb_ratio', 20,longTop=True)
HML.head()

In [ ]:
import itertools
import statsmodels.api as sm
from statsmodels import regression,stats
import scipy

data = healthcare_equity_df[['return']] # dataframe
data = data.set_index(healthcare_equity_df.index) # elimilate redundant index (whole universe)
asset_list_sizes = [group[1].size for group in data.groupby(level=0)]

# Spreading the factor portfolio data across all assets for each day
SMB_column = [[SMB.loc[group[0]]] * size for group, size \
              in zip(data.groupby(level=0), asset_list_sizes)]
data['SMB'] = list(itertools.chain(*SMB_column))

HML_column = [[HML.loc[group[0]]] * size for group, size \
              in zip(data.groupby(level=0), asset_list_sizes)]
data['HML'] = list(itertools.chain(*HML_column))
data = sm.add_constant(data.dropna())

In [ ]:
data.head()

## Factor Exposures ($\beta$)

In [ ]:
assets = data.index.levels[1].unique()
Y = [data.xs(asset,level=1)['return'] for asset in assets]
X = [data.xs(asset,level=1)[['SMB','HML','const']] for asset in assets]
reg_results = [regression.linear_model.OLS(y,x).fit().params for y,x in zip(Y,X) if not(x.empty or y.empty)]
indices = [asset for y, x, asset in zip(Y, X, assets) if not(x.empty or y.empty)]
betas = pd.DataFrame(reg_results, index=indices)

In [ ]:
betas.head()

## Factor Premium

In [ ]:
betas = sm.add_constant(betas.drop('const', axis=1))

R = data['return'].mean(axis=0, level=1)

# Second regression step: estimating the risk premia
risk_free_rate = benchmark_df['return'].mean()

final_results = regression.linear_model.OLS(R - risk_free_rate, betas).fit()

final_results.summary()

## Fama-Macbeth Test Conclusion: 
although our individual factors are significant, we have a very low  $R^2$ . What this may suggest is that there is a real link between our factors and the returns of our assets, but that there still remains a lot of unexplained noise!